In [24]:
import ee
ee.Initialize()
from IPython.display import Image

In [56]:
class runiform_raster:
    def generateRan(self, seed):
        state1 = ee.Image(seed).select('Ran_state')
        state2 = state1.bitwiseXor(state1.leftShift(13))
        state3 = state2.bitwiseXor(state2.rightShift(17))
        state4 = state3.bitwiseXor(state3.leftShift(5))
        state5 = state4.multiply(1597334677).mod(4294967296)
        state6 = state4.addBands(state5.divide(4294967296).select([0],['Ran_uniform']))
        return state6.set('Ran_has_seed', True);
    
    def makeSeed(self, img):
        template = ee.Image(img)
        seed0 = ee.Image.pixelCoordinates(template.projection()).int32().add(2147483648).uint32()
        seed1 = seed0.select(0).bitwiseAnd(65535).leftShift(16).add(seed0.select(1).bitwiseAnd(65535))
        seed2 = seed1.add(seed1.eq(0).multiply(0x7FFF7FFF)).uint32()
        seed3 = seed2.select([0],['Ran_state']).updateMask(template.mask())
        for i in range(10):
          seed3 = self.generateRan(seed3)
        return seed3
  
    def __call__(self, img):
        img = ee.Image(img)
        return ee.Image(ee.Algorithms.If(img.get('Ran_'), self.generateRan(img), self.makeSeed(img)))


In [57]:
asset_dir = 'users/rheilmayr/chile_sim/'
template = ee.Image(asset_dir + 'luq2')
regions = ee.FeatureCollection('ft:1RfoT4UoTohqmJJrNkMT5Jyp95SahGk5f9V355SnH')
reg = regions.filterMetadata('COD_REG', 'equals', '05')
runi = runiform_raster()
rand_img = runi(template)
rand_img.getInfo()

{'bands': [{'crs': 'EPSG:32719',
   'crs_transform': [30.0,
    0.0,
    -136428.67143306672,
    0.0,
    -30.0,
    6619164.6107107885],
   'data_type': {'max': 4294967295,
    'min': 0,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'Ran_state'},
  {'crs': 'EPSG:32719',
   'crs_transform': [30.0,
    0.0,
    -136428.67143306672,
    0.0,
    -30.0,
    6619164.6107107885],
   'data_type': {'max': 1.0,
    'min': 0.0,
    'precision': 'float',
    'type': 'PixelType'},
   'id': 'Ran_uniform'}],
 'properties': {'Ran_has_seed': True},
 'type': 'Image'}

In [38]:
Image(url=rand_img.clip(reg).getThumbUrl({'min':0, 'max':1}))


In [52]:
rand_img.get('Ran_has_seed').getInfo()

True